In [243]:
import pandas as pd
import numpy as np
import statistics

from string import punctuation
from re import sub

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag

import spacy
nlp = spacy.load("en_core_web_md")

In [244]:
stop_words = list(set(stopwords.words('english')))
total_df = pd.read_csv('/Users/madke/Documents/AAPECSMadeline.csv', index_col = 0)

cats = total_df[pd.notnull(total_df.stressDescription)]
catu = total_df[pd.notnull(total_df.unpleasantEventDescription)] 
catu = catu[~(catu['unpleasantEventDescription'] == 'none')]
catu = catu[~(catu['unpleasantEventDescription'] == 'none ')]
catp = total_df[pd.notnull(total_df.pleasantEventDescription)]

stress_text = list(cats['stressDescription'])
unpleasant_text = list(catu['unpleasantEventDescription'])
pleasant_text = list(catp['pleasantEventDescription'])

IC_sample = stress_text + pleasant_text + unpleasant_text
pID = list(cats['participantID']) + list(catp['participantID']) + list(catu['participantID'])

nlp = spacy.load("en_core_web_md")

def prettify (pos_tagged):    
    p_2 = ""
    for z in pos_tagged:
        if (z[0].lower() != 'i') and (('VB' in z[1]) or ('NN' in z[1])):
            p_2= p_2 + " " + str(z[0].lower())
    if len(p_2) == 0:
        for z in pos_tagged:
            if len(z[0]) > 4 and (z[0]) != ' none':
                p_2= p_2 + " " + str(z[0].lower())
    return(p_2)

translator = str.maketrans('','', sub('\#', '', punctuation))

IC_samplew = []
for i in IC_sample:
    n = i.split()
    sub = ""
    for z in n:
        z = z.lower()
        s = z.translate(str.maketrans(translator))
        sub = sub + " " + s
    IC_samplew.append(sub)

In [178]:
class FancyString(str):
    
    def __init__(self, sent):
        self.txt = sent
        
    def is_passive(self):
        z = nltk.word_tokenize(self)
        meh = nltk.pos_tag(z)
        f = prettify(meh)
        f = nltk.word_tokenize(f)
        f = nltk.pos_tag(f)
        q = 0
        countpass = 0
        passlist = []
        for n in f:
           # print(n)
            if (n[1] == 'VBN'):
                q = f.index(n)
                for z in meh[q-1]:
                    if ('have' not in meh[q-1] and 'has' not in meh[q-1] and 'had' not in meh[q-1] and 'VB' in z):
                        if ((meh[q-1][0] + ' ' + n[0]) not in passlist):
                            self.passive = 1
                            return(self.passive)
        self.passive = 0
        return(self.passive)

    def wordcount(self):
        split = self.split()
        self.length = (len(split))
        return(self.length)

    def me_count (self):    
        p_2 = ""
        split = self.split()
        first_single = ['i', 'me', 'mine', 'myself']
        first_s = 0
        for z in split:
            if (z.lower() in first_single):
                first_s += 1
        self.me_count = first_s
        return(self.me_count)

    def ad_count(self):
        z = nltk.word_tokenize(self)
        meh = nltk.pos_tag(z)
        adj = 0
        for i in meh:
            if (i[1] == 'RB') or (i[1] == 'JJ'):
                adj += 1
        self.ad_count = adj
        return(self.ad_count)
    
    def spacy_vec(self):
        v = str(self)
        v = nlp(v).vector
        self.spacy_vec = v
        return(self.spacy_vec)

In [245]:
fancy_strs = [FancyString(i) for i in IC_samplew if i != 'none']

In [247]:
df_constructed = pd.DataFrame({
    
    'pID': pID,
    'passive': [i.is_passive() for i in fancy_strs],
    'wc': [i.wordcount() for i in fancy_strs],
    'mec': [i.me_count() for i in fancy_strs],
    'adc': [i.ad_count() for i in fancy_strs],
    'val': [1] * len(fancy_strs),
    'group': ['stress']*len(stress_text) +['pleasant']*len(pleasant_text)+['unpleasant']*len(unpleasant_text)
    
})

In [248]:
df_const_short = df_constructed.groupby(['pID', 'group']).agg('mean')

In [249]:
formal_vals = pd.pivot_table(df_const_short, index=['pID'], columns=['group'], aggfunc=np.sum, fill_value=0)
formal_vals = formal_vals.reset_index()
formal_vals.columns = ['pID', 'adc_stress', 'adc_pleasant', 'adc_unpleasant', 'mec_stress', 'mec_pleasant', 'mec_unpleasant', 'passive_stress', 'passive_pleasant',
       'passive_unpleasant', 'val_stress', 'val_pleasant', 'val_unpleasant', 'wc_stress', 'wc_pleasant', 'wc_unpleasant']
formal_vals.to_csv('/users/madke/documents/092520_aapecsformalvals.csv')

In [181]:
vector = [i.spacy_vec() for i in fancy_strs]

In [191]:
vec_df = pd.DataFrame(np.vstack(vector))
vec_df.to_csv('/users/madke/documents/092620_aapecsdims.csv')

In [241]:
(total_df[total_df['unpleasantEventDescription'].str.match('none ', na = False)])

,participantID,beginTimeEMA,finishTimeEMA,stressDescription,unpleasantEventDescription,pleasantEventDescription
2173,133,2017-03-08 18:29:32,2017-03-08 18:37:06,NaN,none since last assessment,All 3 kids played a game other than electronics
2197,133,2017-03-13 18:14:04,2017-03-13 18:19:43,NaN,none,My son has already showered which is a huge ac...


In [250]:
print(len(stress_text)) 
print(len(pleasant_text))
print(len(unpleasant_text))

2446
4144
1016
